In [1]:
import gc
import pandas as pd
import numpy as np
import os

## 데이터 로드

In [3]:
path = 'data/'

In [4]:
%%time
## 데이터 로드
## order_products__prior.csv
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order':np.uint8,
                                                                                      'reordered': bool})
## orders.csv
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number':np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

CPU times: user 9.58 s, sys: 776 ms, total: 10.4 s
Wall time: 10.4 s


## 데이터 전처리 #1

In [58]:
%%time

## orders.csv 에서 eval_set이 prior인것만 골라내기
orders = orders.loc[orders.eval_set == 'prior', :]

## orders.csv 에서 'order_id', 'user_id' 칼럼만 추출
orders_user = orders[['order_id', 'user_id']]

CPU times: user 168 ms, sys: 28 ms, total: 196 ms
Wall time: 195 ms


In [59]:
orders.head(1)

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   

   days_since_prior_order  
0                     NaN

In [60]:
orders_user.head(1)

order_id  user_id
0   2539329        1

In [61]:
order_prior.head(1)

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1       True

In [62]:
%time
## 추출한 orders_user 와 order_prior 를 order_id로 병합하기 

## 이유 : order_prior와 orders는 위 head(1) 로 알 수 있듯 order_id로 연결돼 있지만 서로 다른 정보(칼럼)을 갖고있다.
##       따라서 합쳐서 정보를 병합한다.

labels = pd.merge(order_prior, orders_user, on='order_id')
labels.shape

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


(32434489, 5)

In [63]:
%%time
## 'user_id', 'product_id' 를 기준으로 같은 중복을 제거 
## 즉 1명의 유저가 1개의 상품을 여러번 구매한 정보를 제거한다.
## 이유 : labels이기 때문에 유저가 구매한 적이 있는지 아닌지 판단하기 위함.

labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()
labels.shape

CPU times: user 7.24 s, sys: 1.14 s, total: 8.38 s
Wall time: 8.38 s
